In [1]:
#! git clone https://github.com/ALjone/INF368-Final-Project

# A dependency of the preprocessing for BERT inputs
!pip install -q -U tensorflow-text
!pip install -q tf-models-official
!pip install numpy
!pip install pandas


In [2]:
import sys
import numpy as np
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')
os.chdir('..')
from Bert import Bert


In [3]:
data_name = "imdb"
num_classes = 2

data_path = 'data/'+data_name
files = os.listdir(data_path)
labeled_files = [data_path+"/"+file for file in files if "train_labeled" in file]
test_path = data_path+"/test.csv"

In [4]:
batch_size = 4
seed = 0
learning_rate = 5e-5
epochs=5
results=pd.DataFrame(columns=["n_per_class", "accuracy"])


In [ ]:
for n_per_class in [5,10,25,50]:
    data_file = ""
    result = {"n_per_class":n_per_class}
    bert = Bert(num_classes = 2, random_state = seed)
    for file in labeled_files:
        if f"train_labeled_{n_per_class}.csv" in file:
            data_file = file
            break
    print(data_file)
    bert.train_from_path(data_file,learning_rate=learning_rate,batch_size=batch_size,epochs=epochs)
    performance = bert.evaluate_from_path(test_path)
    result["accuracy"] = performance[1]
    results = results.append(result,ignore_index=True)

    


BERT model selected           : https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3


2021-11-11 16:06:22.949889: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10800 MB memory:  -> device: 0, name: Tesla K80, pci bus id: 0000:06:00.0, compute capability: 3.7
2021-11-11 16:06:26.100253: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


data/imdb/train_labeled_5.csv
Epoch 1/5
3/3 [==============================] - 21s 261ms/step - loss: 0.8798 - accuracy: 0.5000
Epoch 2/5
3/3 [==============================] - 1s 242ms/step - loss: 0.8023 - accuracy: 0.6000
Epoch 3/5
3/3 [==============================] - 1s 242ms/step - loss: 0.8143 - accuracy: 0.5000
Epoch 4/5
3/3 [==============================] - 1s 242ms/step - loss: 0.3634 - accuracy: 0.9000
Epoch 5/5
500/500 [==============================] - 18s 34ms/step - loss: 0.7131 - accuracy: 0.5780
BERT model selected           : https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3
data/imdb/train_labeled_10.csv
Epoch 1/5
5/5 [==============================] - 21s 291ms/step - loss: 0.6899 - accuracy: 0.6500
Epoch 2/5
5/5 [==============================] - 1s 280ms/step - loss: 0.8311 - accuracy: 0.5000
Epoch 3/5
5/5 [==============================] - 1s 279ms/step - loss: 0.

In [6]:
results

,n_per_class,accuracy
0,5.0,0.566
1,10.0,0.594
2,25.0,0.696
3,50.0,0.688


In [9]:
if not os.path.exists('results'):
      os.mkdir('results')
result_path = f'results/{data_name}'
if not os.path.exists(result_path):
      os.mkdir(result_path)
results.to_csv(f"{result_path}/bert_results.csv", index=False)